In [30]:
import numpy as np

# Data given
x_data = np.array([-1.0, -0.91, -0.82, -0.70, -0.51, -0.33, -0.1, 0.01, 0.22, 0.34, 0.42, 0.63, 0.78, 0.90])
I_data = np.array([0.39, -0.94, -0.42, -0.24, 1.29, -0.61, -0.1, 0.89, 0.47, -0.89, -0.82, 0.90, 0.99, -0.02])
sigma_data = np.array([0.30, 0.30, 0.30, 0.25, 0.20, 0.20, 0.15, 0.10, 0.10, 0.15, 0.20, 0.25, 0.30, 0.30])

# Model for intensity
def I_theo(x, a0, a1, a2, k):
    return a0 + a1 * np.cos(k * x) + a2 * np.sin(k * x)

# Chi-square function (no scipy)
def chi_square(params, x_data, I_data, sigma_data, k):
    a0, a1, a2 = params
    residuals = (I_data - I_theo(x_data, a0, a1, a2, k)) / sigma_data
    return np.sum(residuals**2)

# Fitting the data using normal equations
def fit_data(k):
    # Design matrix
    A_matrix = np.column_stack([np.ones(len(x_data)), np.cos(k * x_data), np.sin(k * x_data)])
    
    # Least squares solution
    params = np.linalg.lstsq(A_matrix, I_data, rcond=None)[0]
    
    # Covariance matrix (error propagation)
    residuals = I_data - np.dot(A_matrix, params)
    sigma2 = np.sum(residuals**2) / (len(x_data) - 3)  # Degrees of freedom
    covariance_matrix = sigma2 * np.linalg.inv(np.dot(A_matrix.T, A_matrix))
    
    return params, covariance_matrix

# Fit for k=10 and k=11
params_k10, covariance_k10 = fit_data(10)
params_k11, covariance_k11 = fit_data(11)

print(f"Best params for k=10: {params_k10}")
print(f"Best params for k=11: {params_k11}")

# Reduced chi-square for p-value
def chi_square_reduced(params, x_data, I_data, sigma_data, k):
    a0, a1, a2 = params
    residuals = (I_data - I_theo(x_data, a0, a1, a2, k)) / sigma_data
    chi_sq = np.sum(residuals**2)
    dof = len(x_data) - 3  # 3 params (a0, a1, a2)
    chi_sq_red = chi_sq / dof
    p_value = 1 - np.exp(-chi_sq_red / 2)  # Approximation for p-value
    return p_value

# P-values for k=10 and k=11
p_value_k10 = chi_square_reduced(params_k10, x_data, I_data, sigma_data, 10)
p_value_k11 = chi_square_reduced(params_k11, x_data, I_data, sigma_data, 11)

print(f"P-value for k=10: {p_value_k10}")
print(f"P-value for k=11: {p_value_k11}")

# Transform params into A and x0
def transform_parameters(a1, a2, k):
    A = np.sqrt(a1**2 + a2**2)
    x0 = np.arctan2(a2, a1) / k  # arctan2 for full angle range
    return A, x0

# Transform for k=10 and k=11
A_k10, x0_k10 = transform_parameters(params_k10[1], params_k10[2], 10)
A_k11, x0_k11 = transform_parameters(params_k11[1], params_k11[2], 11)

print(f"For k=10: A = {A_k10}, x0 = {x0_k10}")
print(f"For k=11: A = {A_k11}, x0 = {x0_k11}")

# Error propagation for x0
def error_propagation(C, a1, a2, k):
    # Partial derivatives
    d_x0_da1 = -a2 / (a1**2 + a2**2)
    d_x0_da2 = a1 / (a1**2 + a2**2)
    # Error calculation
    error_x0 = np.sqrt(d_x0_da1**2 * C[1, 1] + d_x0_da2**2 * C[2, 2] + 2 * d_x0_da1 * d_x0_da2 * C[1, 2])
    return error_x0

# Error in x0 for k=10 and k=11
error_x0_k10 = error_propagation(covariance_k10, params_k10[1], params_k10[2], 10)
error_x0_k11 = error_propagation(covariance_k11, params_k11[1], params_k11[2], 11)

print(f"Error in x0 for k=10: {error_x0_k10}")
print(f"Error in x0 for k=11: {error_x0_k11}")

# Monte Carlo for x0 error propagation
def monte_carlo_error_propagation(N_MC, x_data, I_data, sigma_data, k):
    x0_values = []
    for _ in range(N_MC):
        I_synthetic = I_theo(x_data, *params_k10, 10) + np.random.normal(0, sigma_data)
        params_MC, _ = fit_data(k)
        A_MC, x0_MC = transform_parameters(params_MC[1], params_MC[2], k)
        x0_values.append(x0_MC)
    # Mean and std dev of x0
    x0_mean = np.mean(x0_values)
    x0_std = np.std(x0_values)
    return x0_mean, x0_std

# Run Monte Carlo for k=10
x0_mean_k10, x0_std_k10 = monte_carlo_error_propagation(100000, x_data, I_data, sigma_data, 10)
print(f"Monte Carlo Mean of x0 for k=10: {x0_mean_k10}")
print(f"Monte Carlo Std Dev of x0 for k=10: {x0_std_k10}")


Best params for k=10: [0.16963434 0.6601444  0.84474137]
Best params for k=11: [0.16238851 0.61895647 0.56924979]
P-value for k=10: 0.36645394927342667
P-value for k=11: 0.9072826751275394
For k=10: A = 1.0720907653330856, x0 = 0.0907453530598205
For k=11: A = 0.8409235625391566, x0 = 0.06759901491188999
Error in x0 for k=10: 0.0791997214297118
Error in x0 for k=11: 0.22866889207827737
Monte Carlo Mean of x0 for k=10: 0.09074535305982048
Monte Carlo Std Dev of x0 for k=10: 1.3877787807814457e-17
